# Oracle Jaycar Part detection proof of concept

This is a part detector for use within jaycar stores

# Base directories and variables


In [1]:
import os
base_dir = '/tmp/oracle'
image_bucket = 'oracle-reference-images'

%mkdir /tmp/oracle

## Prepare Dataset
these are jpg images, Sagemaker built in expects dataset formatted in Record IO so we need to prepare the RECORDIO files
these images are in our `~/oracle/images/AZ0--9` directories

In [2]:
!pip install python-mnist
import os
import boto3
import io
import sagemaker
import numpy as np

#get role 
from sagemaker import get_execution_role
role = get_execution_role()
s3 = boto3.client('s3')

os.chdir(base_dir)
os.mkdir('images')
objlist = s3.list_objects(Bucket=image_bucket)
files = []

for c in objlist['Contents']:
    k = c['Key']
    d = 'images/' + k.split('/')[0]
    print('downloading {} into {}'.format(k,d))
    try:
        os.mkdir(d)
    except Exception:
        pass
    
    s3.download_file(image_bucket, k,'images/'+k)


downloading PA3566/1539557366_0.jpg into images/PA3566
downloading PA3566/1539557366_1.jpg into images/PA3566
downloading PA3566/1539557367_2.jpg into images/PA3566
downloading PA3566/1539557376_0.jpg into images/PA3566
downloading PA3566/1539557376_1.jpg into images/PA3566
downloading PA3566/1539557377_2.jpg into images/PA3566
downloading PA3566/1539557381_0.jpg into images/PA3566
downloading PA3566/1539557381_1.jpg into images/PA3566
downloading PA3566/1539557382_2.jpg into images/PA3566
downloading PA3566/1539557385_0.jpg into images/PA3566
downloading PA3566/1539557385_1.jpg into images/PA3566
downloading PA3566/1539557385_2.jpg into images/PA3566
downloading PA3566/1539557391_0.jpg into images/PA3566
downloading PA3566/1539557391_1.jpg into images/PA3566
downloading PA3566/1539557392_2.jpg into images/PA3566
downloading PA3566/1539557394_0.jpg into images/PA3566
downloading PA3566/1539557395_1.jpg into images/PA3566
downloading PA3566/1539557395_2.jpg into images/PA3566
downloadin

downloading PS0804/1539559146_0.jpg into images/PS0804
downloading PS0804/1539559147_1.jpg into images/PS0804
downloading PS0804/1539559147_2.jpg into images/PS0804
downloading PS0804/1539559149_0.jpg into images/PS0804
downloading PS0804/1539559149_1.jpg into images/PS0804
downloading PS0804/1539559150_2.jpg into images/PS0804
downloading PS0804/1539559154_0.jpg into images/PS0804
downloading PS0804/1539559155_1.jpg into images/PS0804
downloading PS0804/1539559155_2.jpg into images/PS0804
downloading PS0804/1539559159_0.jpg into images/PS0804
downloading PS0804/1539559160_1.jpg into images/PS0804
downloading PS0804/1539559160_2.jpg into images/PS0804
downloading PS0804/1539559163_0.jpg into images/PS0804
downloading PS0804/1539559163_1.jpg into images/PS0804
downloading PS0804/1539559163_2.jpg into images/PS0804
downloading PS0804/1539559165_0.jpg into images/PS0804
downloading PS0804/1539559166_1.jpg into images/PS0804
downloading PS0804/1539559166_2.jpg into images/PS0804
downloadin

In [3]:
import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
print(im2rec)
%env IM2REC=$im2rec
%env BASE_DIR=$base_dir
os.chdir(base_dir)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/tools/im2rec.py
env: IM2REC=/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/tools/im2rec.py
env: BASE_DIR=/tmp/oracle


# Split the databse into test and train
this uses IM2REC to make different listings of what files

In [6]:
%%bash
cd $BASE_DIR
python $IM2REC --list --recursive --test-ratio=0.3 --train-ratio=0.7 oracle images
ls *.lst
head *.lst

PA3566 0
PP0800 1
PS0804 2
PT3020 3
oracle_test.lst
oracle_train.lst
==> oracle_test.lst <==
143	2.000000	PS0804/1539559136_2.jpg
16	0.000000	PA3566/1539557395_1.jpg
173	2.000000	PS0804/1539559173_2.jpg
187	3.000000	PT3020/1539558495_1.jpg
5	0.000000	PA3566/1539557377_2.jpg
72	1.000000	PP0800/1539559030_0.jpg
223	3.000000	PT3020/1539558541_1.jpg
81	1.000000	PP0800/1539559039_0.jpg
97	1.000000	PP0800/1539559053_1.jpg
191	3.000000	PT3020/1539558499_2.jpg

==> oracle_train.lst <==
98	1.000000	PP0800/1539559054_2.jpg
120	1.000000	PP0800/1539559083_0.jpg
172	2.000000	PS0804/1539559172_1.jpg
71	1.000000	PP0800/1539559028_2.jpg
153	2.000000	PS0804/1539559149_0.jpg
238	3.000000	PT3020/1539558574_1.jpg
237	3.000000	PT3020/1539558573_0.jpg
270	3.000000	PT3020/1539558764_0.jpg
0	0.000000	PA3566/1539557366_0.jpg
157	2.000000	PS0804/1539559155_1.jpg


In [7]:
%%bash 

cd $BASE_DIR
python $IM2REC --num-thread=4 --pass-through oracle_test.lst images
python $IM2REC --num-thread=4 --pass-through oracle_train.lst images
ls *.rec

Creating .rec file from /tmp/oracle/oracle_test.lst in /tmp/oracle
time: 0.0007343292236328125  count: 0
Creating .rec file from /tmp/oracle/oracle_train.lst in /tmp/oracle
time: 0.0007202625274658203  count: 0
oracle_test.rec
oracle_train.rec


### Upload rec files to S3 for safekeeping


In [8]:
import boto3
import sagemaker
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()

#INFO:sagemaker:Created S3 bucket: sagemaker-ap-southeast-2-349967396867

train_path = sagemaker_session.upload_data(path=base_dir+'/oracle_train.rec',key_prefix='oracle/train')
train_path = sagemaker_session.upload_data(path=base_dir+'/oracle_test.rec',key_prefix='oracle/test')


INFO:sagemaker:Created S3 bucket: sagemaker-ap-southeast-2-349967396867


#### Setup your Environment
Permissions and environment variables

Here we set up the linkage and authentication to AWS services. There are three parts to this:

1. The roles used to give learning and hosting access to your data. This will automatically be obtained from the role used to start the notebook
2. The S3 bucket that you want to use for training and model data
3. The Amazon sagemaker image classification docker image which need not be changed

In [15]:
%%time
import boto3
import re
import os
import time

from time import gmtime, strftime
from sagemaker import get_execution_role

# 1. Obtaining the role you already configured for Sagemaker when you setup
# your Instance notebook (https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html)
role = get_execution_role()

# 2. The S3 Bucket that will store the dataset and the trained model
# It was already defined above, while we uploaded the RecordIO files to the S3 bucket.

# 3. Select the correct Docker image with the Image Classification algorithm
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/image-classification:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/image-classification:latest',
              'ap-southeast-2': '544295431143.dkr.ecr.ap-southeast-2.amazonaws.com/image-classification:latest'}
training_image = containers[boto3.Session().region_name]
print(training_image)

544295431143.dkr.ecr.ap-southeast-2.amazonaws.com/image-classification:latest
CPU times: user 41.1 ms, sys: 3.58 ms, total: 44.7 ms
Wall time: 858 ms


## hyperparameters

In [19]:
# The algorithm supports multiple network depth (number of layers). They are 18, 34, 50, 101, 152 and 200
# For this training, we will use 152 layers
num_layers = 152

# we need to specify the input image shape for the training data
image_shape = "3,240,480"

# we also need to specify the number of training samples in the training set
num_training_samples = 273

# specify the number of output classes
num_classes = 4

# batch size for training
mini_batch_size = 5

# number of epochs
epochs = 40

# learning rate
learning_rate = 0.00001

# Since we are using transfer learning, we set use_pretrained_model to 1 so that weights can be 
# initialized with pre-trained weights
use_pretrained_model = 1

# Training algorithm/optimizer. Default is SGD
optimizer = 'sgd'


### Sagemaker Job
using below params and hyper params. send a job off


In [20]:
dataset_prefix='oracle'
# create unique job name 
job_name_prefix = 'oracle'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

training_params = {}

# Here we set the reference for the Image Classification Docker image, stored on ECR (https://aws.amazon.com/pt/ecr/)
training_params["AlgorithmSpecification"] = {
    "TrainingImage": training_image,
    "TrainingInputMode": "File"
}

# The IAM role with all the permissions given to Sagemaker
training_params["RoleArn"] = role

# Here Sagemaker will store the final trained model
training_params["OutputDataConfig"] = {
    "S3OutputPath": 's3://{}/{}/output'.format(default_bucket, job_name_prefix)
}

# This is the config of the instance that will execute the training
training_params["ResourceConfig"] = {
    "InstanceCount": 1,
    "InstanceType": "ml.p2.xlarge",
    "VolumeSizeInGB": 50
}

# The job name. You'll see this name in the Jobs section of the Sagemaker's console
training_params["TrainingJobName"] = job_name

# Here you will configure the hyperparameters used for training your model.
training_params["HyperParameters"] = {
    "image_shape": image_shape,
    "num_layers": str(num_layers),
    "num_training_samples": str(num_training_samples),
    "num_classes": str(num_classes),
    "mini_batch_size": str(mini_batch_size),
    "epochs": str(epochs),
    "learning_rate": str(learning_rate),
    "use_pretrained_model": str(use_pretrained_model),
    "optimizer": optimizer
}

# Training timeout
training_params["StoppingCondition"] = {
    "MaxRuntimeInSeconds": 360000
}

# The algorithm currently only supports fullyreplicated model (where data is copied onto each machine)
training_params["InputDataConfig"] = []

# Please notice that we're using application/x-recordio for both 
# training and validation datasets, given our dataset is formated in RecordIO

# Here we set training dataset
# Training data should be inside a subdirectory called "train"
training_params["InputDataConfig"].append({
    "ChannelName": "train",
    "DataSource": {
        "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": 's3://{}/{}/train/'.format(default_bucket, dataset_prefix),
            "S3DataDistributionType": "FullyReplicated"
        }
    },
    "ContentType": "application/x-recordio",
    "CompressionType": "None"
})

# Here we set validation dataset
# Validation data should be inside a subdirectory called "validation"
training_params["InputDataConfig"].append({
    "ChannelName": "validation",
    "DataSource": {
        "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": 's3://{}/{}/test/'.format(default_bucket, dataset_prefix),
            "S3DataDistributionType": "FullyReplicated"
        }
    },
    "ContentType": "application/x-recordio",
    "CompressionType": "None"
})

print('Training job name: {}'.format(job_name))
print('\nInput Data Location: {}'.format(training_params['InputDataConfig'][0]['DataSource']['S3DataSource']))

Training job name: oracle-2018-10-15-04-35-29

Input Data Location: {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-ap-southeast-2-349967396867/oracle/train/', 'S3DataDistributionType': 'FullyReplicated'}


## Model creation
You'll create your model in four steps:
  1. First you'll submit a job for sagemaker to train your model using your dataset uploaded to S3
  2. Second you'll pack the job output into model ready to be used
  3. Then you'll create an Endpoint Configuration, which is the metadata used by Sagemaker to deploy your model
  4. Finally you'll deploy your model using the Endpoint Configuration
  

In [21]:
sagemaker = boto3.client('sagemaker')

sagemaker.create_training_job(**training_params)

status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print('Training job current status: {}'.format(status))

try:
    # wait for the job to finish and report the ending status
    sagemaker.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
    training_info = sagemaker.describe_training_job(TrainingJobName=job_name)
    status = training_info['TrainingJobStatus']
    print("Training job ended with status: " + status)
except:
    print('Training failed to start')
     # if exception is raised, that means it has failed
    message = sagemaker.describe_training_job(TrainingJobName=job_name)['FailureReason']
    print('Training failed with the following error: {}'.format(message))

Training job current status: InProgress
Training job ended with status: Completed


In [23]:
%%time
import boto3
from time import gmtime, strftime

use_pretrained_model=False
bucket = default_bucket

model_name="oracle" + time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
print(model_name)

info = sagemaker.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': training_image,
    'ModelDataUrl': model_data,
}

try:
    create_model_response = sagemaker.create_model(
        ModelName = model_name,
        ExecutionRoleArn = role,
        PrimaryContainer = primary_container)
    print(create_model_response['ModelArn'])
except Exception as e:
    print(e)

oracle-2018-10-15-05-11-57
s3://sagemaker-ap-southeast-2-349967396867/oracle/output/oracle-2018-10-15-04-35-29/output/model.tar.gz
arn:aws:sagemaker:ap-southeast-2:349967396867:model/oracle-2018-10-15-05-11-57
CPU times: user 14.9 ms, sys: 70 µs, total: 14.9 ms
Wall time: 481 ms


#### create endpoint config for model.
support configuring rest endpoints in hosting with multiple models eg for A/B testing purposes. customers create an endpoint configuration, that describes the distrution of traffic across the models, either split shadowed or sampled in some way.

hence the:

In [24]:
from time import gmtime, strftime

timestamp = time.strftime('%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-epc-' + timestamp
endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.c4.2xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

Endpoint configuration name: oracle-epc-2018-10-15-05-13-45
Endpoint configuration arn:  arn:aws:sagemaker:ap-southeast-2:349967396867:endpoint-config/oracle-epc-2018-10-15-05-13-45


## Deploy model.


In [25]:
%%time
import time

timestamp = time.strftime('%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = job_name_prefix + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sagemaker.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))


# get the status of the endpoint
response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))


# wait until the status has changed
sagemaker.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)


# print the status of the endpoint
endpoint_response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response['EndpointStatus']
print('Endpoint creation ended with EndpointStatus = {}'.format(status))

if status != 'InService':
    raise Exception('Endpoint creation failed.')

Endpoint name: oracle-ep-2018-10-15-05-14-16
EndpointArn = arn:aws:sagemaker:ap-southeast-2:349967396867:endpoint/oracle-ep-2018-10-15-05-14-16
EndpointStatus = Creating
Endpoint creation ended with EndpointStatus = InService
CPU times: user 161 ms, sys: 0 ns, total: 161 ms
Wall time: 5min 32s


##### Test your model
Let's recapitulate. We've just trained an Image Classifier model using `data`. What does that mean? It means that now, we have an 'WebService' accessible by the endpoint we just deployed, that has the power to classify 4 different types of objects

lets try:


In [ ]:
%%

In [ ]:
import json
import numpy as np
from io import BytesIO
%matplotlib inline

runtime = boto3.Session().client(service_name='sagemaker-runtime') 
object_categories = ['PT3020', 'PS0804','PP0800','PA3566']

_, axarr = plt.subplots(1, 4, figsize=(20,12))
col = 0
for i in range(4):
    
    # Load the image bytes
    img = open(base_dir+'{}.jpg'.format(i)).read()
    
    # Call your model for predicting which object appears in this image.
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType='application/x-image', 
        Body=bytearray(img)
    )
    # read the prediction result and parse the json
    result = response['Body'].read()
    result = json.loads(result)
    
    # which category has the highest confidence?
    pred_label_id = np.argmax(result)
    
    # Green when our model predicted correctly, otherwise, Red
    text_color = 'red'
    if object_categories[pred_label_id] == test_categories[i]:
        text_color = 'green'

    # Render the text for each image/prediction
    output_text = '%s (%f)' %(object_categories[pred_label_id], result[pred_label_id] )
    axarr[col].text(0, 0, output_text, fontsize=15, color=text_color)
    print( output_text )
    
    # Render the image
    img = Image.open(BytesIO(img))
    frame = axarr[col].imshow(img)
    
    col += 1
plt.show()